# Домашнее задание 3

В этом задании напишем простое решение классификации датасета `FashionMNIST`, а затем будем его улучшать с помощью:
- dropout;
- batch normalization;
- LR scheduler;

В конце сохраним модель в файл и убедимся, что этот файл можем в дальнейшем прочитать.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass

torch.manual_seed(987)

In [ ]:
# Загружаем датасет с картинками одежды FashionMNIST
# root- путь
# train- трейновая часть
# download- если отсутствует то загрузить?
# transofrm преобразуем сразу в тензор

train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.8MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 270kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.06MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.28MB/s]


In [ ]:
# тут дело в том, что в датасете есть
# интересный момент, в предыущем шаге мы создавали датасет train_dataset с ToTensor
# тут мы обращаемся к этому объекту через .data к его неоработанному массиву (не теноры)
X_train = train_dataset.data.float()
y_train = train_dataset.targets
X_test = test_dataset.data.float()
y_test = test_dataset.targets

In [ ]:
@dataclass
class TrainConfig:
    lr: float = 1e-4 # было 3, увеличил в 5-10 раз
    total_iterations: int = 100


# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
# на вход функция принимает y_pred и y_true
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1) # y_pred выдает матрицу с вероятностями у каждого класса. Значение в строчке и есть вероятности. Строчек может быть много
    # первую переменную(сами значения) пропускает, записывате только вторую(индексы этих значений). после = ищет максимльное значение в y_pred по СТРОЧКАМ (так как 1)(было бы 0 были было по столбцам)
    correct = (predicted == y_true).float().sum() # проверка что конкретные значения равны y_true. В тип флоат и суммах их
    accuracy = correct / y_true.shape[0]
    return accuracy.item()

## Задание №1

Попробуйте реализовать простой бейзлайн с несколькими слоями:
- Linear
- ReLU
- Linear

Вставьте свою релизацию `SimpleModel` в проверку.
Вам нужно дописать и сдать как `SimpleModel`, так и `train_loop`.

Используйте кросс-энтропию как функцию потерь.

In [ ]:
# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


# разбиение на трейн тест
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)



# Возможно, класс нужно отнаследовать от некого класса из pytorch
class SimpleModel(nn.Module):
  def __init__(self, num_classes: int):
      super().__init__()


      # размерность после первого слоя
      hidden_dim = 512

      self.net = nn.Sequential(
          nn.Linear(in_features= 28*28 , out_features= hidden_dim),
          nn.ReLU(),
          nn.Linear(in_features=hidden_dim, out_features=num_classes)
      )

  def forward(self, x: torch.Tensor):
        # входной вектор мы решейпим до тензора и этих тензоров столько сколько картинок
        # так как это подается в первый слой, он должен иметь такую же размерность
        x = x.reshape((-1, 28 * 28))

        return self.net(x)



  # """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

  #   Можете так же писать/рисовать accuracy в процессе обучения.
  #   Например, каждые 10 итераций или даже каждую итерацию.
  #   """






# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)



def train_loop(
    model: SimpleModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    # создается оптимайзер
    optimizer = optim.SGD(params=model.parameters(), lr= config.lr)

    losses = []
    loss_fn = nn.CrossEntropyLoss()
    # Пройдемся 2000 раз по всем данным
    for i in tqdm.trange(config.total_iterations):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        # 1 -- ИДЕТ ЗАПУСК МОДЕЛИ ВВЕРХУ!!! Проходит forward pass записывается в аутпут
        outputs = model(X_train)

        # Вычисляем лосс
        loss = loss_fn(outputs, y_train) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())


        # ЦИКЛ ДЛЯ ОЦЕНКИ МОДЕЛИ
        if (i + 1) % 10 == 0: # проверяем что i кратен 10 (делится ли на 10 без остатка)
            model.eval() # переключаем на режим оценки inference mode
            with torch.no_grad(): # отключает подсчеты градиента для все вычислений внутри блока with
                train_acc = calculate_accuracy(outputs, y_train) # outputs-предсказания сравнивниваем с y_train
                val_outputs = model(X_val) # запускам валидационные данные в модель ?????? Это ещё не вероятности
                val_loss = F.cross_entropy(val_outputs, y_val)  #функция потерь. Сравниваем с y_val
                val_acc = calculate_accuracy(val_outputs, y_val) # считаем точность
            model.train() # включаем обратно BatchNorm и дропауты
            print(
                f"Iteration {i+1}: Train Loss = {loss.item():.4f}, Tra  in Acc = {train_acc:.4f}, Val Loss = {val_loss.item():.4f}, Val Acc = {val_acc:.4f}"
            )



    return losses


model = SimpleModel(num_classes=10)  # Создаём объект!
config = TrainConfig()  # Создаём объект TrainConfig
losses = train_loop(model, X_train, y_train, X_val, y_val, config)

 10%|█         | 10/100 [00:20<02:49,  1.89s/it]

tensor([[ -4.4852,  -7.5957,  -0.1859,  ...,  28.3525,   8.4731,  -6.3709],
        [ -2.0865,   0.0304,   4.6188,  ...,  -0.9735,  -3.1472,  -0.7510],
        [  1.9511,  -1.7994,  -0.9848,  ...,  14.0024,   2.6934,  29.5953],
        ...,
        [  3.0517,   2.2798,   6.7182,  ...,   4.9355,  -0.6009,  -2.9287],
        [ 24.6324,  -0.3651,   6.7563,  ...,  -7.6748,  14.8810, -25.5284],
        [  3.0767,  12.7606,  -3.4705,  ...,  -5.1555,  -5.8980,  -4.1368]],
       grad_fn=<AddmmBackward0>)
Iteration 10: Train Loss = 1.4895, Tra  in Acc = 0.6277, Val Loss = 1.4135, Val Acc = 0.6387


 15%|█▌        | 15/100 [00:30<02:54,  2.06s/it]


KeyboardInterrupt: 

NameError: name 'outputs' is not defined

## Задание №2
Какое максимальное значение метрики accuracy удалось получить в процессе обучения?

Округлите до 3 значений после запятой


## Задание №3
Добавьте один `dropout` слой в вашу модель.

_Подумайте, что может поменяться при перестановке ReLU и Dropout слоев местами._

In [ ]:
## Возможно, класс нужно отнаследовать от некого класса из pytorch
# class DropoutModel:
#    hidden_dim = 512


# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)


torch.manual_seed(987)
config = TrainConfig()


# Возможно, класс нужно отнаследовать от некого класса из pytorch
class DropoutModel(nn.Module):
  def __init__(self, num_classes: int, dropout_rate: float = 0.5):
      super().__init__()


      # размерность после первого слоя
      hidden_dim = 512

      self.net = nn.Sequential(
          nn.Linear(in_features= 28*28 , out_features= hidden_dim),
          nn.ReLU(),
          nn.Dropout(p=dropout_rate),
          nn.Linear(in_features=hidden_dim, out_features=num_classes)
      )

  def forward(self, x: torch.Tensor):
        # входной вектор мы решейпим до тензора и этих тензоров столько сколько картинок
        # так как это подается в первый слой, он должен иметь такую же размерность
        x = x.reshape((-1, 28 * 28))

        return self.net(x)



  # """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

  #   Можете так же писать/рисовать accuracy в процессе обучения.
  #   Например, каждые 10 итераций или даже каждую итерацию.
  #   """






# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)



def train_loop(
    model: DropoutModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    # создается оптимайзер
    optimizer = optim.SGD(params=model.parameters(), lr= config.lr)

    losses = []
    loss_fn = nn.CrossEntropyLoss()
    # Пройдемся 2000 раз по всем данным
    for i in tqdm.trange(config.total_iterations):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        # 1 -- ИДЕТ ЗАПУСК МОДЕЛИ ВВЕРХУ!!! Проходит forward pass записывается в аутпут
        outputs = model(X_train)

        # Вычисляем лосс
        loss = loss_fn(outputs, y_train) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())


        # ЦИКЛ ДЛЯ ОЦЕНКИ МОДЕЛИ
        if (i + 1) % 10 == 0: # проверяем что i кратен 10 (делится ли на 10 без остатка)
            model.eval() # переключаем на режим оценки inference mode
            with torch.no_grad(): # отключает подсчеты градиента для все вычислений внутри блока with
                train_acc = calculate_accuracy(outputs, y_train) # outputs-предсказания сравнивниваем с y_train
                val_outputs = model(X_val) # запускам валидационные данные в модель ?????? Это ещё не вероятности
                val_loss = F.cross_entropy(val_outputs, y_val)  #функция потерь. Сравниваем с y_val
                val_acc = calculate_accuracy(val_outputs, y_val) # считаем точность
            model.train() # включаем обратно BatchNorm и дропауты
            print(
                f"Iteration {i+1}: Train Loss = {loss.item():.4f}, Tra  in Acc = {train_acc:.4f}, Val Loss = {val_loss.item():.4f}, Val Acc = {val_acc:.4f}"
            )



    return losses


model = DropoutModel(num_classes=10, dropout_rate=0.5)  # Создаём объект!
config = TrainConfig()  # Создаём объект TrainConfig
losses = train_loop(model, X_train, y_train, X_val, y_val, config)

 10%|█         | 10/100 [00:27<03:47,  2.53s/it]

Iteration 10: Train Loss = 2.4228, Tra  in Acc = 0.6079, Val Loss = 1.3508, Val Acc = 0.7053


 20%|██        | 20/100 [00:51<03:15,  2.44s/it]

Iteration 20: Train Loss = 1.6807, Tra  in Acc = 0.6489, Val Loss = 0.9928, Val Acc = 0.7375


 30%|███       | 30/100 [01:14<02:57,  2.54s/it]

...

Iteration 80: Train Loss = 0.9758, Tra  in Acc = 0.7200, Val Loss = 0.6713, Val Acc = 0.7863


 90%|█████████ | 90/100 [03:36<00:25,  2.57s/it]

Iteration 90: Train Loss = 0.9429, Tra  in Acc = 0.7257, Val Loss = 0.6548, Val Acc = 0.7895


100%|██████████| 100/100 [04:00<00:00,  2.41s/it]

Iteration 100: Train Loss = 0.9269, Tra  in Acc = 0.7290, Val Loss = 0.6410, Val Acc = 0.7913


NameError: name 'y_pred' is not defined

In [ ]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №4
Какое максимальное значение accuracy получилось в ходе обучения модели?

Округлите до 3х знаков после запятой и отправьте в ЛМС.

## Задание №5

Добавьте `BatchNorm` в вашу модель.
Отправьте в ЛМС реализацию.

Стоит ли делать BatchNorm до ReLU или после него?
Это дискуссионный вопрос, чаще всего применяют сначала нелинейность, затем Batch Norm.
Один из аргументов: при таком подходе данные на выходе будут иметь среднее 0 - что и ожидают люди, когда добавляют нормализацию.

_[Дискуссия на Reddit](https://www.reddit.com/r/MachineLearning/comments/67gonq/d_batch_normalization_before_or_after_relu/)_

Для определенности в этом задании будем следовать такому порядку: сначала ReLU, затем Batch Norm.

In [ ]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class BatchNormModel:
    hidden_dim = 512
    ...

In [ ]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

In [ ]:
## Возможно, класс нужно отнаследовать от некого класса из pytorch
# class DropoutModel:
#    hidden_dim = 512


# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)


torch.manual_seed(987)
config = TrainConfig()


# Возможно, класс нужно отнаследовать от некого класса из pytorch
class BatchNormModel(nn.Module):
  def __init__(self, num_classes: int, dropout_rate: float = 0.5):
      super().__init__()


      # размерность после первого слоя
      hidden_dim = 512

      self.net = nn.Sequential(
          nn.Linear(in_features= 28*28 , out_features= hidden_dim),
          nn.BatchNorm1d(num_features=hidden_dim),
          nn.ReLU(),
          nn.Dropout(p=dropout_rate),
          nn.Linear(in_features=hidden_dim, out_features=num_classes)
      )

  def forward(self, x: torch.Tensor):
        # входной вектор мы решейпим до тензора и этих тензоров столько сколько картинок
        # так как это подается в первый слой, он должен иметь такую же размерность
        x = x.reshape((-1, 28 * 28))

        return self.net(x)



  # """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

  #   Можете так же писать/рисовать accuracy в процессе обучения.
  #   Например, каждые 10 итераций или даже каждую итерацию.
  #   """






# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)



def train_loop(
    model: DropoutModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    # создается оптимайзер
    optimizer = optim.SGD(params=model.parameters(), lr= config.lr)

    losses = []
    loss_fn = nn.CrossEntropyLoss()
    # Пройдемся 2000 раз по всем данным



    for i in tqdm.trange(config.total_iterations):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        # 1 -- ИДЕТ ЗАПУСК МОДЕЛИ ВВЕРХУ!!! Проходит forward pass записывается в аутпут
        outputs = model(X_train)

        # Вычисляем лосс
        loss = loss_fn(outputs, y_train) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())


        # ЦИКЛ ДЛЯ ОЦЕНКИ МОДЕЛИ
        if (i + 1) % 10 == 0: # проверяем что i кратен 10 (делится ли на 10 без остатка)
            model.eval() # переключаем на режим оценки inference mode
            with torch.no_grad(): # отключает подсчеты градиента для все вычислений внутри блока with
                train_acc = calculate_accuracy(outputs, y_train) # outputs-предсказания сравнивниваем с y_train
                val_outputs = model(X_val) # запускам валидационные данные в модель ?????? Это ещё не вероятности
                val_loss = F.cross_entropy(val_outputs, y_val)  #функция потерь. Сравниваем с y_val
                val_acc = calculate_accuracy(val_outputs, y_val) # считаем точность
            model.train() # включаем обратно BatchNorm и дропауты
            print(
                f"Iteration {i+1}: Train Loss = {loss.item():.4f}, Tra  in Acc = {train_acc:.4f}, Val Loss = {val_loss.item():.4f}, Val Acc = {val_acc:.4f}"
            )



    return losses


model = BatchNormModel(num_classes=10, dropout_rate=0.5)  # Создаём объект!
config = TrainConfig()  # Создаём объект TrainConfig
losses = train_loop(model, X_train, y_train, X_val, y_val, config)

## Задание №6
Какое максимальное значение `accuracy` получилось в ходе обучения модели?

Округлите до 3х знаков после запятой.

Результат batch normalization мог не особо порадовать.
Но не спешите с выводами насчет этого слоя!

Попробуйте обучить заново все три модели со значением `lr=1e-2` (в 10 раз больше).
Сравните результаты моделей и сделайте вывод.

## Задание №7
Добавьте `LRscheduler` в вашу модель.

Подробнее про `schedulers` можно почитать в [документации](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

In [ ]:
## Возможно, класс нужно отнаследовать от некого класса из pytorch
# class DropoutModel:
#    hidden_dim = 512


# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


from sklearn.model_selection import train_test_split



from torch.optim.lr_scheduler import StepLR

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)


torch.manual_seed(987)
config = TrainConfig()


# Возможно, класс нужно отнаследовать от некого класса из pytorch
class BatchNormModel(nn.Module):
  def __init__(self, num_classes: int, dropout_rate: float = 0.5):
      super().__init__()


      # размерность после первого слоя
      hidden_dim = 512

      self.net = nn.Sequential(
          nn.Linear(in_features= 28*28 , out_features= hidden_dim),
          nn.BatchNorm1d(num_features=hidden_dim),
          nn.ReLU(),
          nn.Dropout(p=dropout_rate),
          nn.Linear(in_features=hidden_dim, out_features=num_classes)
      )

  def forward(self, x: torch.Tensor):
        # входной вектор мы решейпим до тензора и этих тензоров столько сколько картинок
        # так как это подается в первый слой, он должен иметь такую же размерность
        x = x.reshape((-1, 28 * 28))

        return self.net(x)



  # """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

  #   Можете так же писать/рисовать accuracy в процессе обучения.
  #   Например, каждые 10 итераций или даже каждую итерацию.
  #   """






# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)



def train_loop_with_scheduler(
    model: BatchNormModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    # создается оптимайзер
    optimizer = optim.SGD(params=model.parameters(), lr= config.lr)

    losses = []
    loss_fn = nn.CrossEntropyLoss()
    # Пройдемся 2000 раз по всем данным

      #### Новое: создаем разный ШЕДУЛЕР в зависимости от конфига ####
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)





    for i in tqdm.trange(config.total_iterations):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        # 1 -- ИДЕТ ЗАПУСК МОДЕЛИ ВВЕРХУ!!! Проходит forward pass записывается в аутпут
        outputs = model(X_train)

        # Вычисляем лосс
        loss = loss_fn(outputs, y_train) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()


        # Запомним loss
        losses.append(loss.detach().item())


        # ЦИКЛ ДЛЯ ОЦЕНКИ МОДЕЛИ
        if (i + 1) % 10 == 0: # проверяем что i кратен 10 (делится ли на 10 без остатка)
            model.eval() # переключаем на режим оценки inference mode
            with torch.no_grad(): # отключает подсчеты градиента для все вычислений внутри блока with
                train_acc = calculate_accuracy(outputs, y_train) # outputs-предсказания сравнивниваем с y_train
                val_outputs = model(X_val) # запускам валидационные данные в модель ?????? Это ещё не вероятности
                val_loss = F.cross_entropy(val_outputs, y_val)  #функция потерь. Сравниваем с y_val
                val_acc = calculate_accuracy(val_outputs, y_val) # считаем точность
            model.train() # включаем обратно BatchNorm и дропауты
            print(
                f"Iteration {i+1}: Train Loss = {loss.item():.4f}, Tra  in Acc = {train_acc:.4f}, Val Loss = {val_loss.item():.4f}, Val Acc = {val_acc:.4f}"
            )

        scheduler.step()

    return losses


model = BatchNormModel(num_classes=10, dropout_rate=0.5)  # Создаём объект!
config = TrainConfig()  # Создаём объект TrainConfig
losses = train_loop_with_scheduler(model, X_train, y_train, X_val, y_val, config)

 10%|█         | 10/100 [00:21<03:10,  2.11s/it]

Iteration 10: Train Loss = 2.3504, Tra  in Acc = 0.1232, Val Loss = 2.2887, Val Acc = 0.1385


 20%|██        | 20/100 [00:40<02:28,  1.85s/it]

Iteration 20: Train Loss = 2.3471, Tra  in Acc = 0.1279, Val Loss = 2.2755, Val Acc = 0.1404


 30%|███       | 30/100 [01:01<02:21,  2.02s/it]

...

Iteration 80: Train Loss = 2.3497, Tra  in Acc = 0.1273, Val Loss = 2.2756, Val Acc = 0.1427


 90%|█████████ | 90/100 [02:57<00:18,  1.90s/it]

Iteration 90: Train Loss = 2.3516, Tra  in Acc = 0.1256, Val Loss = 2.2756, Val Acc = 0.1427


100%|██████████| 100/100 [03:16<00:00,  1.97s/it]

Iteration 100: Train Loss = 2.3492, Tra  in Acc = 0.1252, Val Loss = 2.2756, Val Acc = 0.1427


In [ ]:
from torch.optim.lr_scheduler import StepLR


def train_loop_with_scheduler(
    model,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    ...
    scheduler = StepLR(..., step_size=5, gamma=0.1)
    ...

In [ ]:
torch.manual_seed(987)
config = TrainConfig(lr=1e-3)
# Ваш код для обучения и подсчета accuracy

## Задание №8

Поэксперементируйте с параметрами нейронной сети, попробуйте добиться максимальной метрики `accuracy`.

- попробуйте комбинацию Drouput + Batch Normalization и подумайте, как лучше всего раскрыть силу batch normalization (вспомните эксперименты с lr);
- попробуйте подвигать вероятность в Dropout;
- ну, или подержите обучение подольше, поставив больше шагов :)

В ЛМС нужно сдать код класса `ExpModel`.
Вам необходимо выбить accuracy > 80%, чтобы сдать этот пункт.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass

torch.manual_seed(987)





@dataclass
class TrainConfig:
    lr: float = 1e-1 # было 1е-3
    total_iterations: int = 100


# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
# на вход функция принимает y_pred и y_true
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1) # первую переменную(конкретные значения) пропускает, записывате только вторую. после = ищет максимльное значение в y_pred по строчкам (так как 1)(было бы 0 были было по столбцам)
    correct = (predicted == y_true).float().sum() # проверка что конкретные значения равны y_true. В тип флоат и суммах их
    accuracy = correct / y_true.shape[0]
    return accuracy.item()






# Загружаем датасет с картинками одежды FashionMNIST
# root- путь
# train- трейновая часть
# download- если отсутствует то загрузить?
# transofrm преобразуем сразу в тензор

train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

# тут дело в том, что в датасете есть
# интересный момент, в предыущем шаге мы создавали датасет train_dataset с ToTensor
# тут мы обращаемся к этому объекту через .data к его неоработанному массиву (не теноры)
X_train = train_dataset.data.float()
y_train = train_dataset.targets
X_test = test_dataset.data.float()
y_test = test_dataset.targets






## Возможно, класс нужно отнаследовать от некого класса из pytorch
# class DropoutModel:
#    hidden_dim = 512


# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


from sklearn.model_selection import train_test_split



from torch.optim.lr_scheduler import StepLR

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)


torch.manual_seed(987)
config = TrainConfig()

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.8MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 229kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.21MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.8MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass

torch.manual_seed(987)





@dataclass
class TrainConfig:
    lr: float = 1e-1 # было 1е-3
    total_iterations: int = 100


# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
# на вход функция принимает y_pred и y_true
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1) # первую переменную(конкретные значения) пропускает, записывате только вторую. после = ищет максимльное значение в y_pred по строчкам (так как 1)(было бы 0 были было по столбцам)
    correct = (predicted == y_true).float().sum() # проверка что конкретные значения равны y_true. В тип флоат и суммах их
    accuracy = correct / y_true.shape[0]
    return accuracy.item()






# Загружаем датасет с картинками одежды FashionMNIST
# root- путь
# train- трейновая часть
# download- если отсутствует то загрузить?
# transofrm преобразуем сразу в тензор

train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

# тут дело в том, что в датасете есть
# интересный момент, в предыущем шаге мы создавали датасет train_dataset с ToTensor
# тут мы обращаемся к этому объекту через .data к его неоработанному массиву (не теноры)
X_train = train_dataset.data.float()
y_train = train_dataset.targets
X_test = test_dataset.data.float()
y_test = test_dataset.targets






## Возможно, класс нужно отнаследовать от некого класса из pytorch
# class DropoutModel:
#    hidden_dim = 512


# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
import torch.optim
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F
import torch.nn as nn


from sklearn.model_selection import train_test_split



from torch.optim.lr_scheduler import StepLR

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)


torch.manual_seed(987)
config = TrainConfig()





torch.manual_seed(987)
config = TrainConfig()


# Возможно, класс нужно отнаследовать от некого класса из pytorch
class ExpModel(nn.Module):
  def __init__(self, num_classes: int, dropout_rate: float = 0.3):
      super().__init__()


      # размерность после первого слоя
      hidden_dim = 512

      self.net = nn.Sequential(
          nn.Linear(in_features= 28*28 , out_features= hidden_dim),
          nn.ReLU(),
          nn.BatchNorm1d(num_features=hidden_dim),
          nn.Dropout(p=dropout_rate),
          nn.Linear(in_features=hidden_dim, out_features=num_classes)
      )

  def forward(self, x: torch.Tensor):
        # входной вектор мы решейпим до тензора и этих тензоров столько сколько картинок
        # так как это подается в первый слой, он должен иметь такую же размерность
        x = x.reshape((-1, 28 * 28))

        return self.net(x)



  # """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

  #   Можете так же писать/рисовать accuracy в процессе обучения.
  #   Например, каждые 10 итераций или даже каждую итерацию.
  #   """






# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)



def train_loop(
    model: ExpModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    # создается оптимайзер
    optimizer = optim.SGD(params=model.parameters(), lr= config.lr, momentum=0.9)

    losses = []
    loss_fn = nn.CrossEntropyLoss()
    # Пройдемся 2000 раз по всем данным

      #### Новое: создаем разный ШЕДУЛЕР в зависимости от конфига ####
    scheduler = StepLR(optimizer, step_size=20, gamma=0.1)





    for i in tqdm.trange(config.total_iterations):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        # 1 -- ИДЕТ ЗАПУСК МОДЕЛИ ВВЕРХУ!!! Проходит forward pass записывается в аутпут
        outputs = model(X_train)

        # Вычисляем лосс
        loss = loss_fn(outputs, y_train) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()


        # Запомним loss
        losses.append(loss.detach().item())


        # ЦИКЛ ДЛЯ ОЦЕНКИ МОДЕЛИ
        if (i + 1) % 10 == 0: # проверяем что i кратен 10 (делится ли на 10 без остатка)
            model.eval() # переключаем на режим оценки inference mode
            with torch.no_grad(): # отключает подсчеты градиента для все вычислений внутри блока with
                train_acc = calculate_accuracy(outputs, y_train) # outputs-предсказания сравнивниваем с y_train
                val_outputs = model(X_val) # запускам валидационные данные в модель ?????? Это ещё не вероятности
                val_loss = F.cross_entropy(val_outputs, y_val)  #функция потерь. Сравниваем с y_val
                val_acc = calculate_accuracy(val_outputs, y_val) # считаем точность
            model.train() # включаем обратно BatchNorm и дропауты
            print(
                f"Iteration {i+1}: Train Loss = {loss.item():.4f}, Tra  in Acc = {train_acc:.4f}, Val Loss = {val_loss.item():.4f}, Val Acc = {val_acc:.4f}"
            )

        scheduler.step()

    return losses


model = ExpModel(num_classes=10, dropout_rate=0.3)  # Создаём объект!
config = TrainConfig(lr=1e-1)  # Создаём объект TrainConfig
losses = train_loop(model, X_train, y_train, X_val, y_val, config)
torch.save(model.state_dict(), 'model.pt')

 10%|█         | 10/100 [00:26<03:46,  2.52s/it]

Iteration 10: Train Loss = 0.5341, Tra  in Acc = 0.8109, Val Loss = 0.6291, Val Acc = 0.7980


 20%|██        | 20/100 [00:52<03:23,  2.55s/it]

Iteration 20: Train Loss = 0.4313, Tra  in Acc = 0.8443, Val Loss = 0.4365, Val Acc = 0.8465


 30%|███       | 30/100 [01:18<03:07,  2.67s/it]

...

Iteration 80: Train Loss = 0.4057, Tra  in Acc = 0.8529, Val Loss = 0.3900, Val Acc = 0.8588


 90%|█████████ | 90/100 [03:55<00:26,  2.66s/it]

Iteration 90: Train Loss = 0.4048, Tra  in Acc = 0.8536, Val Loss = 0.3899, Val Acc = 0.8588


100%|██████████| 100/100 [04:21<00:00,  2.62s/it]

Iteration 100: Train Loss = 0.4051, Tra  in Acc = 0.8535, Val Loss = 0.3899, Val Acc = 0.8590


In [ ]:
torch.save(model.state_dict(), 'model.pt')
# checkpoint = torch.load('model_501.pt', weights_only=False)
# # model.load_state_dict(checkpoint)

# from google.colab import files
# files.download('model_501.pt')  # Скачиваем файл

In [ ]:
from google.colab import files
files.download('model_501.pt')  # Скачиваем файл

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# torch.manual_seed(987)


# # Ваш код модели и ее обучения при seed = 987
# class ExpModel: ...


# model = ExpModel()
# config = TrainConfig(...)
# train_loop(model, X_train, y_train, X_test, y_test, config)

Наконец, сохраним лучшую модель, чтобы в будущем ее могли взять и использовать, без обучения.

## Задание №9

Напишите код, который сохранит модель в файл `model.pt`.

In [ ]:
# Впоследствии эту модель можно будет загрузить вот так:
model_loaded = ExpModel(num_classes=len(y_test.unique()))
model_loaded.load_state_dict(torch.load("model.pt"))